In [1]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [4]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "1EftloFacjKqS3H6OJJBt9Q5A"
    consumer_secret = "WD8tgpNIIHyJU7LgYp6IPDcSyDlKxR3GIYhKsjEUR9BSEFlOJ1"
    access_token = "1251974096654491648-yd33b23jmgjfdZxcvzKyUkkikjeaPr"
    access_token_secret = "cM2cEghN5HrqfK3xARBCrYq3nYyYWs8DZFDVGvcBj3BdP"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-79.150082, 40.392234, -73.371869, 44.781112,  # New York
                 -164.639405, 58.806859, -144.152365, 71.76871]  # Alaska

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)

{'created_at': 'Mon Apr 20 02:11:10 +0000 2020', 'id': 1252057188207677443, 'id_str': '1252057188207677443', 'text': 'https://t.co/m4Nf72KoTl', 'display_text_range': [0, 0], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 859285964, 'id_str': '859285964', 'name': 'Lucki', 'screen_name': 'merlynluciano', 'location': '10029', 'url': None, 'description': 'undercover brother', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 11, 'friends_count': 310, 'listed_count': 0, 'favourites_count': 107, 'statuses_count': 239, 'created_at': 'Wed Oct 03 06:45:18 +0000 2012', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color

{'created_at': 'Mon Apr 20 02:11:10 +0000 2020', 'id': 1252057190417956867, 'id_str': '1252057190417956867', 'text': '@DeeTheSav_ https://t.co/4UBg8PQRlh', 'display_text_range': [11, 11], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1252056004642189313, 'in_reply_to_status_id_str': '1252056004642189313', 'in_reply_to_user_id': 1542411956, 'in_reply_to_user_id_str': '1542411956', 'in_reply_to_screen_name': 'DeeTheSav_', 'user': {'id': 1010876596230778880, 'id_str': '1010876596230778880', 'name': 'Nece', 'screen_name': 'NeceNuggz', 'location': 'Queens, NY', 'url': None, 'description': 'University of Bridgeport Alumna ⚔️ .22🥂 .Dancer🕺🏽 Instagram: @Necenuggz', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 132, 'friends_count': 123, 'listed_count': 0, 'favourites_count': 569, 'statuses_count': 542, 'created_at': 'Sun Jun 24 13:25:36 +0000 2018', 'utc_offset'

{'created_at': 'Mon Apr 20 02:11:11 +0000 2020', 'id': 1252057194650107906, 'id_str': '1252057194650107906', 'text': 'LMFAOOOOOOO bill', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 947602811918995456, 'id_str': '947602811918995456', 'name': 'NICK HALDEN 💜💛', 'screen_name': 'n1ckh4lden', 'location': 'Midtown East, Manhattan', 'url': 'http://Instagram.com/ridoy4rahman', 'description': '@drake your lyrics do shit for me', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 183, 'friends_count': 280, 'listed_count': 2, 'favourites_count': 22515, 'statuses_count': 23051, 'created_at': 'Sun Dec 31 22:58:10 +0000 2017', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': 

{'created_at': 'Mon Apr 20 02:11:14 +0000 2020', 'id': 1252057204024389634, 'id_str': '1252057204024389634', 'text': 'Ok gross. Who the fuck is writing this shit with Issa?!', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 50179868, 'id_str': '50179868', 'name': 'Brittany♚', 'screen_name': 'britshaniece', 'location': 'Philly/NYC', 'url': None, 'description': '♚AKA♚ ♚IG & Snapchat: britshaniece', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 6166, 'friends_count': 457, 'listed_count': 118, 'favourites_count': 970, 'statuses_count': 339281, 'created_at': 'Wed Jun 24 02:06:30 +0000 2009', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': F

{'created_at': 'Mon Apr 20 02:11:18 +0000 2020', 'id': 1252057222579916802, 'id_str': '1252057222579916802', 'text': 'But Roger stone and Paul manaford should be locked away', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1220003224897368078, 'id_str': '1220003224897368078', 'name': 'Mike', 'screen_name': 'Mike29668613', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 3, 'friends_count': 64, 'listed_count': 0, 'favourites_count': 26, 'statuses_count': 1920, 'created_at': 'Wed Jan 22 15:20:59 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_colo

...

{'created_at': 'Mon Apr 20 02:12:06 +0000 2020', 'id': 1252057423386378242, 'id_str': '1252057423386378242', 'text': '@haley_abraszek if this isn’t us idk what is', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 3124209112, 'in_reply_to_user_id_str': '3124209112', 'in_reply_to_screen_name': 'haley_abraszek', 'user': {'id': 220375743, 'id_str': '220375743', 'name': 'Kelly Koz', 'screen_name': 'KellyKozz', 'location': 'Buffalo ➳ ', 'url': 'http://vsco.co/kellykoz/images/1', 'description': '#AK17', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 783, 'friends_count': 652, 'listed_count': 3, 'favourites_count': 15920, 'statuses_count': 7260, 'created_at': 'Sat Nov 27 16:08:58 +0000 2010', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_tr

{'created_at': 'Mon Apr 20 02:12:08 +0000 2020', 'id': 1252057431443746823, 'id_str': '1252057431443746823', 'text': '@thomaskaine5 Its happening in NY they should also have someone wiping down the handle on shopping carts with bleac… https://t.co/oLeSuN4bXg', 'display_text_range': [14, 140], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': 1251851316135112705, 'in_reply_to_status_id_str': '1251851316135112705', 'in_reply_to_user_id': 860654500431880192, 'in_reply_to_user_id_str': '860654500431880192', 'in_reply_to_screen_name': 'thomaskaine5', 'user': {'id': 4925153687, 'id_str': '4925153687', 'name': 'karlabetz', 'screen_name': 'no2tomfoolery', 'location': 'New York, NY', 'url': None, 'description': 'Live, LOVE, Laugh and always Carpe Diem!!!\nLover of words and facts 🍏🍎, because both are so necessary...', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 11

{'created_at': 'Mon Apr 20 02:12:09 +0000 2020', 'id': 1252057436615249920, 'id_str': '1252057436615249920', 'text': 'it’s considered inclusive language', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2974136289, 'id_str': '2974136289', 'name': 'Brandon', 'screen_name': '__brandonpaul', 'location': 'Toronto, Ontario', 'url': 'https://vsco.co/8rand0', 'description': 'hasta el fuego // 19 // YU', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 548, 'friends_count': 532, 'listed_count': 1, 'favourites_count': 24480, 'statuses_count': 14230, 'created_at': 'Mon Jan 12 05:13:38 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': Fal

{'created_at': 'Mon Apr 20 02:12:09 +0000 2020', 'id': 1252057437584097281, 'id_str': '1252057437584097281', 'text': 'Yes', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 473524022, 'id_str': '473524022', 'name': 'Gabriel', 'screen_name': 'go0ley', 'location': 'New York, NY', 'url': 'https://www.instagram.com/go0ley/', 'description': 'That shirtless guy who loves carrot cake. Specialized CPT/Mobility instructor. They call me Gumby gabe.gooley@gmail.com', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 9139, 'friends_count': 313, 'listed_count': 47, 'favourites_count': 28098, 'statuses_count': 115055, 'created_at': 'Wed Jan 25 02:19:18 +0000 2012', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'l

finished
